# Numerical Gradient Checking

This code discusses a numerical gradient checking method. We compute the gradients of parameters of a 3 Layered Neural Network using Back-propagation Algorithm and then compute gradients using first principles of  derivatives and then compare both the values to ensure that back-propagation is working correctly.

In [6]:
import numpy as np

# Helper Functions

In [7]:
def sigmoid(x):
    """This function computes sigmoid of x where x can be a scalar or a matrix. If x is a matrix, it computes sigmoid of all the elements of x"""
    
    s = 1 / (1+np.exp(-x))
    
    return s

In [8]:
def relu(x):
    """This function computes relu of x where x can be a scalar or a matrix. If x is a matrix, it computes sigmoid of all the elements of x """
    
    s=np.maximum(0,x)
    
    return s

In [10]:
def dictionary_to_vector(parameters):
   """This function converts parameters into a long vector."""

    keys = []
    count = 0
    for key in ["W1", "b1", "W2", "b2", "W3", "b3"]:
        
        new_vector = np.reshape(parameters[key], (-1,1 ))           # Unrolling one by one all parameter matrices.
        keys = keys + [key]*new_vector.shape[0]
        
        if count == 0:
            theta = new_vector                                                  
        else:
            theta = np.concatenate((theta, new_vector), axis=0)     # Concatinating small temporary vectors into long big vector theta.
        count = count + 1

    return theta, keys


In [11]:
def gradients_to_vector(gradients):
    """
    This function converts gradient matrices into a long vector.
    """
    
    count = 0
    for key in ["dW1", "db1", "dW2", "db2", "dW3", "db3"]:
        # flatten parameter
        new_vector = np.reshape(gradients[key], (-1,1))
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
        count = count + 1

    return theta

In [21]:
def vector_to_dictionary(theta):
    """
    Unroll all our parameters dictionary from a single vector satisfying our specific required shape.
    """
    parameters = {}
    parameters["W1"] = theta[:20].reshape((5,4))
    parameters["b1"] = theta[20:25].reshape((5,1))
    parameters["W2"] = theta[25:40].reshape((3,5))
    parameters["b2"] = theta[40:43].reshape((3,1))
    parameters["W3"] = theta[43:46].reshape((1,3))
    parameters["b3"] = theta[46:47].reshape((1,1))

    return parameters

# Forward Propagation

We first perform forward propagation on a 3 Layered Neural Network and compute the Cost. The 2 hidden layers are RELU activated and the output layer is Sigmoid Activated.

In [16]:
def forward_propagation_n(X, Y, parameters):
    """ This function computes forward propagation and cost."""
    
    # retrieve parameters
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    Z1 = np.dot(W1 , X) + b1    # Linear part of first hidden layer.
    A1 = relu(Z1)                       # Activation of first hidden layer.   
    Z2 = np.dot(W2 , A1) + b2  
    A2 = relu(Z2)
    Z3 = np.dot(W3 , A2) + b3
    A3 = sigmoid(Z3)
    
    logprobs = - np.multiply(np.log(A3) , Y) - np.multiply(np.log(1-A3), 1-Y)
    cost = 1./m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache
    

# Back Propagation

In [17]:
def backward_propagation_n( X , Y, cache):
    """This function computes gradient using backpropagation algorithm."""
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y                                                                                # Gradient of Cost w.r.t Z3.
    dW3 = 1./m * np.dot(dZ3, A2.T)                                                 # Gradient of Cost w.r.t W3
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)                 # Gradient of cost w.r.t b3.
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T) * 2
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 4./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

# Gradient Checking Using GradApprox



$gradapprox[i] = \frac{J^{+}_i - J^{-}_i}{2 \varepsilon}$

In [22]:
def gradient_check_n(parameters, gradients, X, Y, epsilon = 1e-7):
    
    parameters_values, _ = dictionary_to_vector(parameters)    # Converts all parameter matrices into a long vector.
    grad = gradients_to_vector(gradients)                                   # Converts all gradient matrices into a long vector. 
    num_parameters = parameters_values.shape[0]                    # Number of total paarameters.
    J_plus = np.zeros((num_parameters, 1))                                # Initializing a long vector of zeros of same shape as parameters_values.
    J_minus = np.zeros((num_parameters, 1))                             # Initializing a long vector of zeros of same shape as parameters_values.
    gradapprox = np.zeros((num_parameters, 1))                        # Initializing a long vector of zeros of same shape as parameters_values.
    
    
    # Computing Grad approx:
    
    for i in range(num_parameters):
        
        # Computing J_plus[i]
        thetaplus = np.copy(parameters_values)                                                                                          # Copying parameters_values to thetaplus vector.
        thetaplus[i][0] = thetaplus[i][0] + epsilon                                                                                        # Computing thetaplus[i] + epsilon.
        J_plus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaplus))                               # Computing J_plus[i]
        
        
        # Computing J_minus[i]
        thetaminus = np.copy(parameters_values)                                                                                       # Copying parameters_values to thetaplus vector.
        thetaminus[i][0] = thetaminus[i][0] - epsilon                                                                                   # Computing thetaminus[i] + epsilon.
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus))                         # Computing J_minus[i]    
        
        # Computing gradapprox[i] using the formula given above.
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)
        
        # Comparing the 2 vectors i.e. grad and gradapprox.
         
    numerator = np.linalg.norm(grad - gradapprox)                                                                             # Norm of difference of 2 vectors.
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)                                              # Norm of difference of 2 vectors.   
    difference = numerator/denominator          
        
    if difference > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference
        

In [23]:
def gradient_check_n_test_case(): 
    np.random.seed(1)
    x = np.random.randn(4,3)
    y = np.array([1, 1, 0])
    W1 = np.random.randn(5,4) 
    b1 = np.random.randn(5,1) 
    W2 = np.random.randn(3,5) 
    b2 = np.random.randn(3,1) 
    W3 = np.random.randn(1,3) 
    b3 = np.random.randn(1,1) 
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}

    
    return x, y, parameters

In [24]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

There is a mistake in the backward propagation! difference = 0.285093156781
